In [1]:
# ANCOVA analysis multiverse
from os import listdir
import numpy as np
import pandas as pd
import numpy.matlib
import matplotlib.pyplot as plt
import pingouin as pg
from statsmodels.formula.api import ols
import statsmodels.api as sm
from Support_Functions import *

C:\Users\dario\anaconda3\envs\stats_eeg\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
C:\Users\dario\anaconda3\envs\stats_eeg\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
# Main directory
MainDir = ""
# Directory with data
DataDir = ""
# Path to masterfile
MasterFile = ""

In [3]:
# Read the masterfile and compare demographics
dataMF = pd.read_csv(MasterFile)
# Code the Gender to Male and Female
dataMF.loc[dataMF['Gender'] == 1, 'Gender'] = 'F'
dataMF.loc[dataMF['Gender'] == 2, 'Gender'] = 'M'
# Code Handedness to Right and Left
dataMF.loc[dataMF['Handedness'] != 2, 'Handedness'] = 'R'
dataMF.loc[dataMF['Handedness'] == 2, 'Handedness'] = 'L'
# Change the type of the columns
dataMF.astype({'Group': 'object',
              'Gender': 'object',
              'age': 'float',
              'education': 'float',
              'illduration': 'float',
              'SPQ_I': 'float',
              'SPQ_II': 'float',
              'SPQ_III': 'float',
              'SANS': 'float',
              'SAPS': 'float',
              'CPZ': 'float',
              'Handedness': 'object'})

# Columns of interest
Col_Int = ['Group', 'EEG_ID', 'Gender', 'age', 'education',
           'illduration','SPQ_I', 'SPQ_II', 'SPQ_III',
           'SANS', 'SAPS', 'CPZ', 'Handedness']
    
# get data for each group
dataMF = dataMF[Col_Int]
PatMF = dataMF.loc[dataMF['Group'] == 'SZPatients']
ConMF = dataMF.loc[dataMF['Group'] == 'Controls']

In [4]:
#------------ compare the demographics
# Age
stats_age = pg.ttest(PatMF['age'], ConMF['age'], paired = False, correction = False)

print(f"Age: p-value = {stats_age.loc['T-test','p-val']}") # no age difference

# Education
stats_edu = pg.ttest(PatMF['education'], ConMF['education'], paired = False,
                     alternative = 'two-sided', correction = False)

print(f"Education: p-value = {stats_edu.loc['T-test','p-val']}") # edu difference

# Gender
_, _, stats_gender = pg.chi2_independence(dataMF, x='Group', y='Gender',
                                          correction = False)

print(f"Gender: p-value = {stats_gender.loc[stats_gender['test'] == 'pearson', 'pval'][0]}") # gender difference

Age: p-value = 0.6043730462955819
Education: p-value = 1.6566940385331386e-05
Gender: p-value = 6.689990251335336e-07


In [5]:
# Get the name of the files that end with .csv in Results
# which are name of the files with the features
File_names = list(filter(lambda x: '.csv' in x, listdir(DataDir)))

In [6]:
# function to calculate the biggest differences

def get_BigDiff(inFile, DataPath, DataMF):
    
    """function to do ANCOVAs
    inFile is the name of file with Features
    DataPath is the path of the data
    DataMF is the masterfile"""
    
    # read data
    dataX = pd.read_csv(DataPath + "\\" + inFile)
    
    # get the EEG data
    DataEEG = dataX.values[:,1:].astype(float)
    
    # initiate p_values, cohen's d
    tmp_p_vals = np.zeros(DataEEG.shape[1])
    tmp_p_vals_int = np.zeros(DataEEG.shape[1])
    tmp_cohensD = np.zeros(DataEEG.shape[1])
    tmp_cohensD_ci = np.zeros(DataEEG.shape[1])
    
    # Masterfile values of interest
    MF_val = DataMF[['Group','education','Gender']].values
    
    # ANCOVA formula
    formula = 'Channel ~ Group : Gender + Group + Gender + education'
    
    # do ANCOVA for each "channel"
    for i in range(DataEEG.shape[1]):
        
        # first create a dataFrame for each channel
        df = pd.DataFrame(data = np.hstack((MF_val,DataEEG[:,i].reshape(len(DataEEG[:,i]),1))),
                          columns = ['Group','education','Gender','Channel']).astype(
                              {'Channel': 'float', 'education': 'float'})
        
        # fit the model
        model = ols(formula, df).fit()
        aov_table = sm.stats.anova_lm(model, typ=2)
        aov_table = etaSquared(aov_table) # get the effect size
        eta_sq = aov_table.loc['Group','eta_sq']    # get eta squared

        # print(aov_table.loc['Group','eta_sq'])
        # print(aov_table.loc['Group','PR(>F)'])
        
        # get the sign of effect size based on group differences
        sign_eff = 1
        if np.mean(df.loc[df['Group'] == 'SZPatients','Channel']) - np.mean(
                df.loc[df['Group'] == 'Controls','Channel']) < 0:
                sign_eff = -1
        
        print(f"Eta-squared: {eta_sq}")
        CohensD = etaSquared2CohensD(eta_sq)
        CohensD *= sign_eff                          # get the sign correct 
        print(f"Cohen's D: {CohensD}")
        
        # get the confidence interval of CohensD
        ci = pg.compute_esci(CohensD, nx=list(df['Group']).count('SZPatients'),
                             ny=list(df['Group']).count('Controls'),
                             eftype='cohen', confidence = 0.95, decimals=4)
        
        tmp_p_vals[i] = aov_table.loc['Group','PR(>F)']
        tmp_p_vals_int[i] = aov_table.loc['Group:Gender','PR(>F)']
        tmp_cohensD[i] = CohensD
        tmp_cohensD_ci[i] = (ci[1] - ci[0])/2 
    
    # correct p-values
    _, pvals_corr = pg.multicomp(tmp_p_vals, method='fdr_bh', alpha = 0.05)
    # get the largest difference, i.e., biggest CohensD
    big_diff_ind = np.argmax(np.abs(tmp_cohensD))
    big_diff_p_vals = pvals_corr[big_diff_ind]
    big_diff_p_vals_int = tmp_p_vals_int[big_diff_ind]
    big_diff_cohensD = tmp_cohensD[big_diff_ind]
    big_diff_cohensD_ci = tmp_cohensD_ci[big_diff_ind]
    big_diff_values = DataEEG[:, big_diff_ind]
        
    return big_diff_p_vals, big_diff_cohensD, big_diff_cohensD_ci, big_diff_p_vals_int, big_diff_values

In [7]:
dataX = pd.read_csv(DataDir + "\\" + File_names[0])
    
# get the EEG data
DataEEG = dataX.values[:,1:].astype(float)
    
    # initiate p_values, cohen's d
tmp_p_vals = np.zeros(DataEEG.shape[1])
tmp_cohensD = np.zeros(DataEEG.shape[1])
tmp_cohensD_ci = np.zeros(DataEEG.shape[1])
    
    # Masterfile values of interest
MF_val = dataMF[['Group','education','Gender']].values
    
    # ANCOVA formula
formula = 'Channel ~ Group : Gender + Group + Gender + education'
    
    # do ANCOVA for each "channel"
for i in range(DataEEG.shape[1]):
        
        # first create a dataFrame for each channel
    df = pd.DataFrame(data = np.hstack((MF_val,DataEEG[:,i].reshape(len(DataEEG[:,i]),1))),
                          columns = ['Group','education','Gender','Channel']).astype(
                              {'Channel': 'float', 'education': 'float'})
        
        # fit the model
    model = ols(formula, df).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    
etaSquared(aov_table)

,sum_sq,df,F,PR(>F),eta_sq
Group,2.539807e+04,1.0,3.843076,0.051406,0.018405
Gender,8.344843e+04,1.0,12.626891,0.000479,0.060473
Group:Gender,3.124845e+03,1.0,0.472832,0.492521,0.002264
education,5.681737e+03,1.0,0.859725,0.354986,0.004117
Residual,1.262278e+06,191.0,NaN,NaN,NaN


In [8]:
# claim some memory to store p_values, cohensD, and confidence interval
p_vals = np.zeros(len(File_names))
p_vals_int = np.zeros(len(File_names))
CohensDs = np.zeros(len(File_names))
CohensDs_ci = np.zeros(len(File_names))

# create dataframe to store data
Features = [feature.replace('.csv','') for feature in File_names]
df_BigDiff_values = pd.DataFrame(data=[], columns = Features)

# loop through all features and calculate biggest differences
for ifeat in range(len(File_names)):
    p_vals[ifeat], CohensDs[ifeat], CohensDs_ci[ifeat], p_vals_int[ifeat]\
        , df_BigDiff_values[Features[ifeat]]\
            = get_BigDiff(File_names[ifeat], DataDir, dataMF)
    

Eta-squared: 0.018219125679927086
Cohen's D: 0.2724497901673206
Eta-squared: 0.021100664271879226
Cohen's D: 0.29363582637576424
Eta-squared: 0.0119644399722849
Cohen's D: 0.22008472204236376
Eta-squared: 0.023475031814487898
Cohen's D: 0.31009260857027315
Eta-squared: 0.019425097287452127
Cohen's D: 0.28149534029117523
Eta-squared: 0.03511048510797131
Cohen's D: 0.38151324249143104
Eta-squared: 0.008223585451377198
Cohen's D: 0.18211835228542564
Eta-squared: 0.012612257899197512
Cohen's D: 0.2260385701118586
Eta-squared: 0.016701724710845616
Cohen's D: 0.2606561747494837
Eta-squared: 0.009138269953754172
Cohen's D: 0.19206819713677073
Eta-squared: 0.024060819577213635
Cohen's D: 0.3140319435563794
Eta-squared: 0.0011643493864860353
Cohen's D: 0.06828489368343349
Eta-squared: 0.02878422944229969
Cohen's D: 0.3443098416729325
Eta-squared: 0.016783530123035954
Cohen's D: 0.26130461464492805
Eta-squared: 0.02023280830817585
Cohen's D: 0.28740653100715263
Eta-squared: 0.0020068207875229456

Eta-squared: 0.002370260040859622
Cohen's D: 0.09748623536878784
Eta-squared: 0.0021371543265533634
Cohen's D: 0.09255769092570924
Eta-squared: 0.0033170045143148365
Cohen's D: 0.11537839746028962
Eta-squared: 0.0008515162364974283
Cohen's D: 0.058386366022966166
Eta-squared: 0.003218901124217505
Cohen's D: 0.11365378847091087
Eta-squared: 0.003120943758748514
Cohen's D: 0.11190557703280328
Eta-squared: 0.0042695891586963484
Cohen's D: 0.13096406707228841
Eta-squared: 0.007435778810193832
Cohen's D: 0.17310671652932172
Eta-squared: 0.0048693605582704105
Cohen's D: 0.13990264116557635
Eta-squared: 0.0012630688052556848
Cohen's D: 0.07112429024288582
Eta-squared: 0.008767584114133257
Cohen's D: 0.18809715123334536
Eta-squared: 0.0003996015155787477
Cohen's D: -0.0399880612575517
Eta-squared: 0.0069779427005058885
Cohen's D: 0.16765412658399947
Eta-squared: 0.022389685651161055
Cohen's D: 0.30267120533658287
Eta-squared: 0.021536532444694105
Cohen's D: 0.2967191511389285
Eta-squared: 0.00

Eta-squared: 0.03217441133910926
Cohen's D: 0.364658843580838
Eta-squared: 0.02470635222158318
Cohen's D: 0.3183219687385874
Eta-squared: 0.025039020122245088
Cohen's D: 0.3205125525917586
Eta-squared: 0.015502001128065105
Cohen's D: 0.25096690733042987
Eta-squared: 0.010895085882130562
Cohen's D: 0.209905657878434
Eta-squared: 0.01278380760982441
Cohen's D: 0.2275904181096573
Eta-squared: 0.017617055597661677
Cohen's D: 0.2678281683280251
Eta-squared: 0.014198460581454105
Cohen's D: 0.24002466912903736
Eta-squared: 0.008834968628482522
Cohen's D: 0.1888250097263508
Eta-squared: 0.004262089349132049
Cohen's D: 0.13084850031587283
Eta-squared: 0.0055141274191031955
Cohen's D: 0.14892550634841842
Eta-squared: 0.004543901294231811
Cohen's D: 0.13512427730880583
Eta-squared: 0.00873866283152095
Cohen's D: 0.18778392118336018
Eta-squared: 0.014620087017480275
Cohen's D: 0.24361449321771908
Eta-squared: 0.01750411761987837
Cohen's D: 0.2669529581208185
Eta-squared: 0.021099972404219326
Cohen

Eta-squared: 0.020006018275713234
Cohen's D: 0.2857581476002353
Eta-squared: 0.00020125849768455045
Cohen's D: 0.028375976516839287
Eta-squared: 0.008027319807902925
Cohen's D: 0.1799141919958179
Eta-squared: 0.014674713026407136
Cohen's D: 0.2440759509029523
Eta-squared: 0.02438200363392496
Cohen's D: 0.31617300732994175
Eta-squared: 0.001207739283342965
Cohen's D: 0.06954709717305176
Eta-squared: 0.002438818227773545
Cohen's D: 0.09888944540792433
Eta-squared: 8.324932104656438e-06
Cohen's D: -0.005770615707089823
Eta-squared: 0.016325425111556678
Cohen's D: 0.25765378221466945
Eta-squared: 0.010713952987560168
Cohen's D: 0.20813442710932287
Eta-squared: 0.021180278975127597
Cohen's D: 0.2942012260714444
Eta-squared: 1.1586140557616339e-06
Cohen's D: 0.0021527799684591263
Eta-squared: 0.015792796584921817
Cohen's D: 0.25334728049307237
Eta-squared: 0.0090182197687681
Cohen's D: 0.1907908610748536
Eta-squared: 0.014420858181783404
Cohen's D: 0.24192446913349383
Eta-squared: 0.00838024

Eta-squared: 0.10368986064515827
Cohen's D: 0.6802506828401875
Eta-squared: 0.0875246116844429
Cohen's D: 0.6194189851575204
Eta-squared: 0.09557286568368088
Cohen's D: 0.6501454208742597
Eta-squared: 0.08716872029134606
Cohen's D: 0.6180378513875103
Eta-squared: 0.06187448406326398
Cohen's D: 0.5136358362679667
Eta-squared: 0.052230980620469256
Cohen's D: 0.46950782122574924
Eta-squared: 0.0693871594055104
Cohen's D: 0.5461161622549768
Eta-squared: 0.06969718268560314
Cohen's D: 0.5474260237621197
Eta-squared: 0.08143707464288336
Cohen's D: 0.5955066583580569
Eta-squared: 0.085264118453416
Cohen's D: 0.6106119384858862
Eta-squared: 0.07062354307957654
Cohen's D: 0.551326557680546
Eta-squared: 0.07192224165859415
Cohen's D: 0.5567617878233795
Eta-squared: 0.07358986415149461
Cohen's D: 0.5636861280644071
Eta-squared: 0.07992054298867238
Cohen's D: 0.589449415692438
Eta-squared: 0.07270522027040262
Cohen's D: 0.5600204490477417
Eta-squared: 0.09983852393124802
Cohen's D: 0.6660684460351

...

Eta-squared: 0.0342381950436733
Cohen's D: 0.376574071340393
Eta-squared: 0.043744506624719535
Cohen's D: 0.4277645449803874
Eta-squared: 0.04644251277376125
Cohen's D: 0.44138178838781095
Eta-squared: 0.031679996522323055
Cohen's D: 0.3617538039081871
Eta-squared: 0.01837956092060835
Cohen's D: 0.2736691009141801
Eta-squared: 0.03093135060995157
Cohen's D: 0.3573157548564622
Eta-squared: 0.04773097132812135
Cohen's D: 0.4477651634168522
Eta-squared: 0.050007526398493055
Cohen's D: 0.4588678176257478
Eta-squared: 0.05246909734114566
Cohen's D: 0.4706359527067966
Eta-squared: 0.07602127598830026
Cohen's D: 0.5736758760711677
Eta-squared: 0.073809190766782
Cohen's D: 0.5645923430953692
Eta-squared: 0.04724449993594705
Cohen's D: 0.4453637772162522
Eta-squared: 0.016948868192712165
Cohen's D: 0.2626106248504143
Eta-squared: 0.002666157013255186
Cohen's D: 0.10340762852384314
Eta-squared: 0.02254596630751031
Cohen's D: 0.3037499750330796
Eta-squared: 0.020501991038407375
Cohen's D: 0.28935

Eta-squared: 0.0008704107873181854
Cohen's D: -0.05903114643194933
Eta-squared: 0.00013137309200751085
Cohen's D: 0.022925126229922137
Eta-squared: 5.2713848246754726e-05
Cohen's D: 0.014521243354897406
Eta-squared: 0.003925296703165442
Cohen's D: -0.125551031481319
Eta-squared: 4.1326239681993806e-05
Cohen's D: 0.012857363277229484
Eta-squared: 0.0048197451751475434
Cohen's D: 0.13918459143673936
Eta-squared: 0.004629066639500895
Cohen's D: 0.13639053561876677
Eta-squared: 3.5437991632927713e-06
Cohen's D: 0.0037650029066919914
Eta-squared: 0.00043565404612530437
Cohen's D: 0.04175375064800606
Eta-squared: 0.009617808514590218
Cohen's D: -0.19709093397527047
Eta-squared: 0.0009460178226663882
Cohen's D: 0.061543923171252915
Eta-squared: 0.0025191328178161
Cohen's D: -0.1005086041060471
Eta-squared: 0.0010454408373780848
Cohen's D: 0.06470038416359754
Eta-squared: 0.022020457025269915
Cohen's D: -0.30010848509724974
Eta-squared: 0.0026366410774379344
Cohen's D: 0.10283212205663735
Eta-

Eta-squared: 0.0003643027835647353
Cohen's D: -0.038180390173462884
Eta-squared: 7.383124254757944e-05
Cohen's D: -0.017185656112286576
Eta-squared: 0.007100627207939466
Cohen's D: 0.16913198019052622
Eta-squared: 0.00023774896264508339
Cohen's D: -0.03084188714434807
Eta-squared: 0.020576971064482204
Cohen's D: -0.2898915547424364
Eta-squared: 0.029792708514009503
Cohen's D: -0.3504715121216996
Eta-squared: 0.00741716812436047
Cohen's D: -0.17288832936288223
Eta-squared: 0.010303016867099888
Cohen's D: -0.20406150148138594
Eta-squared: 0.030976567853062617
Cohen's D: -0.35758517458041833
Eta-squared: 0.03044535610494357
Cohen's D: -0.35440870403246044
Eta-squared: 0.006399764551172338
Cohen's D: -0.16051149916228694
Eta-squared: 0.0038731091981396197
Cohen's D: -0.12471036042475775
Eta-squared: 0.017127648042505522
Cohen's D: -0.26401603308505217
Eta-squared: 0.015182297109842908
Cohen's D: -0.2483252122339686
Eta-squared: 0.000326767846469458
Cohen's D: -0.03615935058032505
Eta-squar

Eta-squared: 1.69110606720547e-05
Cohen's D: 0.008224681552708075
Eta-squared: 0.0011614180052703586
Cohen's D: 0.06819878196865846
Eta-squared: 0.0052818707696474405
Cohen's D: -0.14573835549208514
Eta-squared: 0.0002736156449473149
Cohen's D: 0.03308718972402819
Eta-squared: 0.005524096867311405
Cohen's D: 0.14906082008728813
Eta-squared: 6.381701982341223e-06
Cohen's D: 0.005052422274007318
Eta-squared: 0.0020608427901684678
Cohen's D: 0.09088671264379451
Eta-squared: 0.005750357693288492
Cohen's D: -0.1521001716168645
Eta-squared: 0.0032085120633730305
Cohen's D: 0.11346963910296873
Eta-squared: 0.005565009259631154
Cohen's D: 0.1496148641914285
Eta-squared: 3.629473900629566e-06
Cohen's D: 0.0038102425506826585
Eta-squared: 0.008541166160738004
Cohen's D: -0.1856313135108761
Eta-squared: 0.0022814846550758745
Cohen's D: 0.09563894039529228
Eta-squared: 0.00792529443348516
Cohen's D: 0.17875800952178775
Eta-squared: 8.734413063159138e-05
Cohen's D: 0.01869243272505251
Eta-squared: 

Eta-squared: 0.007099224257144799
Cohen's D: 0.16911515123269497
Eta-squared: 0.0016949403823843727
Cohen's D: 0.08240917625640674
Eta-squared: 0.0030930516357546636
Cohen's D: 0.11140284174318524
Eta-squared: 0.009543365520732334
Cohen's D: 0.19631932040241185
Eta-squared: 0.00995401675026492
Cohen's D: 0.20054022423571072
Eta-squared: 0.0007694233750292177
Cohen's D: 0.05549831864905936
Eta-squared: 0.004757146875433017
Cohen's D: 0.13827343190605887
Eta-squared: 0.006619030384209509
Cohen's D: 0.16325604270283442
Eta-squared: 0.007199539357642637
Cohen's D: 0.17031439877064483
Eta-squared: 0.00010567052875384065
Cohen's D: -0.020560320638993657
Eta-squared: 0.006895326977071755
Cohen's D: 0.16665176422072484
Eta-squared: 5.166228492958058e-05
Cohen's D: -0.014375667505776287


In [9]:
# Add a column with Group and save biggest difference
datatmp = pd.read_csv(DataDir + "\\" + File_names[0])
df_BigDiff_values['Group'] = datatmp['Group']
df_BigDiff_values.to_csv("R_Values_of_Biggest_Differences_ANCOVA.csv")

# create data frame to save the biggest differences
df_BigDiff = pd.DataFrame(
    data = np.vstack((p_vals,CohensDs,CohensDs_ci,p_vals_int)).transpose(),
    index = Features, columns = ['p-vals','CohensDs','CohensDs_ci','p-vals-int'])
df_BigDiff.to_csv("R_CohenD_for_ANCOVAs_int.csv")

C:\Users\dario\AppData\Local\Temp\ipykernel_5648\2036853323.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_BigDiff_values['Group'] = datatmp['Group']


In [10]:
# Display number of significant differences
print("Total number of features:",len(df_BigDiff['p-vals']))
print("Number of features with significant differences:",len(np.where(df_BigDiff['p-vals']<0.05)[0]))
print("Number of features with significant group:gender:",len(np.where(df_BigDiff['p-vals-int']<0.05)[0]))

Total number of features: 194
Number of features with significant differences: 69
Number of features with significant group:gender: 20


In [11]:
df_BigDiff = pd.read_csv("CohenD_for_ANCOVAs_int.csv", index_col = 0)
sig_d = np.abs(df_BigDiff.iloc[np.where(df_BigDiff['p-vals']<0.05)[0]]['CohensDs'].values)
max(sig_d)

1.0373791377559125

In [12]:
len(np.where(df_BigDiff.iloc[np.where(df_BigDiff['p-vals']<0.05)[0]]['CohensDs'].values<0)[0])

24